In [1]:
import pandas as pd
import os
import seaborn as sns
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import random
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
draft_path = os.path.join(os.getcwd(), 'FantasyDraft.xlsx')
df = pd.read_excel(draft_path, header=None, index_col=None)
df.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Order,Name,Posititon,Team,GP,FG%,FGM,FGA,FT%,FTM,FTA,3PM,PPG,REB,AST,STL,BLK,TO,OVERALL
2,NaN,1,Victor Wembanyama,C,SA,70,0.473,8.8,18.6,0.802,4.5,5.6,2,24.1,11.1,4.4,1.4,3.9,3.4,NaN
3,NaN,2,Nikola Jokic,C,DEN,75,0.585,10.4,17.8,0.819,4.8,5.8,1.1,26.7,12.1,9.1,1.4,0.9,3.1,NaN
4,NaN,3,Luka Doncic,"PG,SG",DAL,68,0.491,11.3,23,0.792,7.6,9.7,4.1,34.4,9.3,9.9,1.4,0.5,3.8,NaN
5,NaN,4,Shai Gilgeous-Alexander,PG,OKC,73,0.524,10.2,19.5,0.88,8.1,9.2,1.4,29.9,5.1,6.5,1.9,0.9,2.4,NaN
6,NaN,5,Giannis Antetokounmpo,"PF,C",MIL,71,0.614,11.8,19.2,0.671,7,10.5,0.6,31.1,11.1,6.7,1.2,1.1,3.4,NaN
7,NaN,6,Joel Embiid,C,PHI,62,0.518,9.4,18.2,0.871,9,10.3,1.3,29.2,10.1,4.1,1,1.5,3.5,NaN
8,NaN,7,Anthony Davis,"PF,C",LAL,67,0.559,9.5,17,0.802,5.8,7.3,0.4,25.2,12.1,3.6,1.2,2.3,2.1,NaN
9,NaN,8,Tyrese Haliburton,"PG,SG",IND,69,0.483,7.5,15.6,0.862,3.1,3.5,3,21.1,3.9,11,1.4,0.7,2.5,NaN


In [3]:
df.drop(columns=[0, 1], inplace=True)
df.columns = df.iloc[1].values
df.drop(index=[0, 1], inplace=True)
df = df.drop(columns=df.columns[-1])
df.reset_index(drop=True, inplace=True)
df.head()

,Name,Posititon,Team,GP,FG%,FGM,FGA,FT%,FTM,FTA,3PM,PPG,REB,AST,STL,BLK,TO
0,Victor Wembanyama,C,SA,70,0.473,8.8,18.6,0.802,4.5,5.6,2,24.1,11.1,4.4,1.4,3.9,3.4
1,Nikola Jokic,C,DEN,75,0.585,10.4,17.8,0.819,4.8,5.8,1.1,26.7,12.1,9.1,1.4,0.9,3.1
2,Luka Doncic,"PG,SG",DAL,68,0.491,11.3,23,0.792,7.6,9.7,4.1,34.4,9.3,9.9,1.4,0.5,3.8
3,Shai Gilgeous-Alexander,PG,OKC,73,0.524,10.2,19.5,0.88,8.1,9.2,1.4,29.9,5.1,6.5,1.9,0.9,2.4
4,Giannis Antetokounmpo,"PF,C",MIL,71,0.614,11.8,19.2,0.671,7,10.5,0.6,31.1,11.1,6.7,1.2,1.1,3.4


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Name       227 non-null    object
 1   Posititon  201 non-null    object
 2   Team       201 non-null    object
 3   GP         201 non-null    object
 4   FG%        201 non-null    object
 5   FGM        201 non-null    object
 6   FGA        201 non-null    object
 7   FT%        201 non-null    object
 8   FTM        201 non-null    object
 9   FTA        201 non-null    object
 10  3PM        201 non-null    object
 11  PPG        201 non-null    object
 12  REB        201 non-null    object
 13  AST        201 non-null    object
 14  STL        201 non-null    object
 15  BLK        201 non-null    object
 16  TO         201 non-null    object
dtypes: object(17)
memory usage: 30.3+ KB


In [5]:
df = df.dropna()
df = df.reset_index(drop=True)
df.head()

,Name,Posititon,Team,GP,FG%,FGM,FGA,FT%,FTM,FTA,3PM,PPG,REB,AST,STL,BLK,TO
0,Victor Wembanyama,C,SA,70,0.473,8.8,18.6,0.802,4.5,5.6,2,24.1,11.1,4.4,1.4,3.9,3.4
1,Nikola Jokic,C,DEN,75,0.585,10.4,17.8,0.819,4.8,5.8,1.1,26.7,12.1,9.1,1.4,0.9,3.1
2,Luka Doncic,"PG,SG",DAL,68,0.491,11.3,23,0.792,7.6,9.7,4.1,34.4,9.3,9.9,1.4,0.5,3.8
3,Shai Gilgeous-Alexander,PG,OKC,73,0.524,10.2,19.5,0.88,8.1,9.2,1.4,29.9,5.1,6.5,1.9,0.9,2.4
4,Giannis Antetokounmpo,"PF,C",MIL,71,0.614,11.8,19.2,0.671,7,10.5,0.6,31.1,11.1,6.7,1.2,1.1,3.4


In [6]:
df[['GP', 'FG%', 'FGM', 'FGA', 'FT%', 'FTM', 'FTA', '3PM', 'PPG', 'REB', 'AST', 'STL', 'BLK', 'TO']] = df[['GP', 'FG%', 'FGM', 'FGA', 'FT%', 'FTM','FTA', '3PM', 'PPG', 'REB', 'AST', 'STL', 'BLK', 'TO']].apply(pd.to_numeric)

In [7]:
fig = go.Figure()
fig.add_trace(go.Box(y=df['GP'].values, name=df['GP'].name))

In [8]:
fig = px.histogram(df, x="GP")
fig.update_layout(
    xaxis = dict(
        rangeslider = dict(
            visible = True,
            autorange = True
        )
    )
)
fig.show()

In [9]:
df['GP'].describe()

count    201.000000
mean      68.189055
std        7.244590
min       28.000000
25%       67.000000
50%       69.000000
75%       73.000000
max       79.000000
Name: GP, dtype: float64

In [10]:
fig = go.Figure()

for col in df[['FG%','FT%', '3PM', 'PPG', 'REB', 'AST', 'STL', 'BLK', 'TO']]:
  fig.add_trace(go.Box(y=df[col].values, name=df[col].name))
  
fig.show()

In [11]:
cols = ['FG%','FT%', '3PM', 'PPG', 'REB', 'AST', 'STL', 'BLK', 'TO']
fig = make_subplots(rows=3, cols=3, subplot_titles=cols)

for i in range(len(cols)):
    fig.add_trace(
    px.histogram(df, cols[i])['data'][0],
    row = (i // 3) + 1, col = (i % 3) + 1
)

fig.update_layout(showlegend=False);
fig.show()

In [12]:
df['Posititon'] = df['Posititon'].str.replace(',', ' ')
df['Posititon'].head(10)

0        C
1        C
2    PG SG
3       PG
4     PF C
5        C
6     PF C
7    PG SG
8    SF PF
9    SG SF
Name: Posititon, dtype: object

In [13]:
df['Posititon'].unique()

array(['C', 'PG SG', 'PG', 'PF C', 'SF PF', 'SG SF', 'SG SF PF', 'PF',
       'SG', 'SF', 'SF PF C'], dtype=object)

In [14]:
positions = []
unique_positions = []
for i in df['Posititon']:
    for j in i.split():
        positions.append(j)
        if j not in unique_positions:
            unique_positions.append(j)
position_counts = {}
for pos in unique_positions:
    position_counts.update({str(pos): positions.count(pos)})


In [15]:
position_counts.keys()

dict_keys(['C', 'PG', 'SG', 'PF', 'SF'])

In [16]:
fig = px.pie(values=position_counts.values(), names=position_counts.keys(), title = 'Position Distribution')
fig.show()

In [17]:
sc = MinMaxScaler()
df[['3PM', 'PPG', 'REB', 'REB', 'AST', 'STL', 'BLK', 'TO']] = sc.fit_transform(df[['3PM', 'PPG', 'REB', 'REB', 'AST', 'STL', 'BLK', 'TO']])
df.head()

,Name,Posititon,Team,GP,FG%,FGM,FGA,FT%,FTM,FTA,3PM,PPG,REB,AST,STL,BLK,TO
0,Victor Wembanyama,C,SA,70,0.473,8.8,18.6,0.802,4.5,5.6,0.416667,0.629496,0.821429,0.394495,0.6875,1.000000,0.804878
1,Nikola Jokic,C,DEN,75,0.585,10.4,17.8,0.819,4.8,5.8,0.229167,0.723022,0.910714,0.825688,0.6875,0.210526,0.731707
2,Luka Doncic,PG SG,DAL,68,0.491,11.3,23.0,0.792,7.6,9.7,0.854167,1.000000,0.660714,0.899083,0.6875,0.105263,0.902439
3,Shai Gilgeous-Alexander,PG,OKC,73,0.524,10.2,19.5,0.880,8.1,9.2,0.291667,0.838129,0.285714,0.587156,1.0000,0.210526,0.560976
4,Giannis Antetokounmpo,PF C,MIL,71,0.614,11.8,19.2,0.671,7.0,10.5,0.125000,0.881295,0.821429,0.605505,0.5625,0.263158,0.804878


In [18]:
draft_dict = df.set_index('Name').T.to_dict('list');

C:\Users\atliu\AppData\Local\Temp\ipykernel_20004\2308467308.py:1: UserWarning:

DataFrame columns are not unique, some columns will be omitted.



In [19]:
draft_dict_expand = {}
for name in draft_dict.keys():
    for pos in draft_dict[name][0].split(' '):
        draft_dict_expand.update({name + '_' + pos: {'pos': str(pos),
                                                     'team': draft_dict[name][1],
                                                     'gp': draft_dict[name][2],
                                                     'fg': draft_dict[name][3],
                                                     'fgm': draft_dict[name][4],
                                                     'fga': draft_dict[name][5],
                                                     'ft': draft_dict[name][6],
                                                     'ftm': draft_dict[name][7],
                                                     'fta': draft_dict[name][8],
                                                     '3pm': draft_dict[name][9],
                                                     'ppg': draft_dict[name][10],
                                                     'reb': draft_dict[name][11],
                                                     'ast': draft_dict[name][12],
                                                     'stl': draft_dict[name][13],
                                                     'blk': draft_dict[name][14],
                                                     'to': draft_dict[name][15]}})

In [22]:
def group_power(draft_data ,group, w_3PM = 1., w_PPG = 1., w_AST = 1., w_STL = 1., w_BLK = 1., w_TO =1.):
    """ Returns, Power - 3PM - PPG - AST - STL - BLK - TO
    """
    _3pm = 0.
    ppg = 0.
    ast = 0.
    stl = 0.
    blk = 0.
    to = 0.
    for player in group:
        _3pm = _3pm + draft_data[player]["3pm"] / len(group)
        ppg = ppg + draft_data[player]["ppg"] / len(group)
        ast = ast + draft_data[player]["ast"] / len(group)
        stl = stl + draft_data[player]["stl"] / len(group)
        blk = blk + draft_data[player]["blk"] / len(group)
        to = to + draft_data[player]["to"] / len(group)
    power = w_3PM * _3pm + w_PPG * ppg + w_AST * ast + w_STL * stl + w_BLK * blk + w_TO * to

    return [power, _3pm, ppg, ast, stl, blk, to]

In [23]:
def check_dublicate(group):
    players = [player.split("_")[0] for player in group]
    return len(players) != len(set(players))
def group_positions(group):
    return [player.split("_")[1] for player in group]

In [24]:
def group_validity(group, req_positions = [], req = "min"):
    if not check_dublicate(group):
        T = 1
        gp = group_positions(group)
        if req == "equal":
            T =  T*(gp.sort() == req_positions.sort())
        elif req == "max":
            for pos in req_positions:
                T = T*(req_positions.count(pos) >= gp.count(pos))
        else:
            for pos in req_positions:
                T = T*(req_positions.count(pos) <= gp.count(pos))
        if T ==1:
            return True
        else:
            return False
    else:
        return False

In [25]:
# Pick intervals
intervals = np.array([[1, 10], [29, 35], [29, 35], [63, 70], [63, 70],  [90, 100], [90, 100], [130, 145], [130, 145],  [160, 180],  [160, 180], [190, 200]])
intervals

array([[  1,  10],
       [ 29,  35],
       [ 29,  35],
       [ 63,  70],
       [ 63,  70],
       [ 90, 100],
       [ 90, 100],
       [130, 145],
       [130, 145],
       [160, 180],
       [160, 180],
       [190, 200]])

In [26]:
pick_lists = []
players = df["Name"].values
for interval in intervals:
    pick_lists.append(players[interval[0]: interval[1]])
for i in range(len(pick_lists)):
    players = []
    for player in pick_lists[i]:
        pos = draft_dict[player][0].split(" ")
        for p in pos:
            players = np.append(players, (player + "_" + str(p)))
    pick_lists[i] = players
print(len(pick_lists))

12


In [27]:
player_list = list(draft_dict_expand.keys())

In [28]:
group_size = 12
pre_group = []
comp = ["C", "C", "C"]
groups = np.empty((0, group_size))
group = np.append(pre_group, random.sample(player_list,group_size - len(pre_group)))
groups = np.append(groups, [group],axis=0)
g_power = group_power(draft_data=draft_dict_expand, group=group)

In [29]:
for i in range(10000):
    random_group = []
    random_group = np.append(pre_group, random_group)
    for i in range(0, len(pick_lists)):
        random_group = np.append(random_group, random.sample(sorted(pick_lists[i]), 1))
    if group_validity(group=random_group, req_positions=comp, req="max"):
        random_group_power = group_power(draft_data=draft_dict_expand, group=random_group)
        if random_group_power[0] > g_power[0]:
            g_power = random_group_power
            groups = np.append(groups, [random_group], axis=0)
print(groups.shape[0])

6


In [30]:
if groups.shape[0] >= 15:
    top_count = int(groups.shape[0] / 3)
else:
    top_count = groups.shape[0]
top_groups = groups[-top_count:]
group_stats = np.empty((0,len(g_power)))
for group in top_groups:
    group_stats = np.append(group_stats, [group_power(draft_data=draft_dict_expand, group=group)], axis=0)

In [31]:
for i in top_groups:
    print(i, "\n")


['Franz Wagner_PF' 'Tyrese Haliburton_PG' 'Keon Ellis_SF'
 'Tyrese Maxey_PG' 'Anthony Davis_C' 'Draymond Green_PF'
 'DeMar DeRozan_SF' 'Kristaps Porzingis_PF' 'Bruce Brown_SG'
 'Karl-Anthony Towns_C' 'Tari Eason_SF' 'Mikal Bridges_PF'] 

['Luka Doncic_PG' 'Dejounte Murray_SG' 'Paolo Banchero_PF'
 'Zach LaVine_SG' 'Cam Thomas_SG' 'Jalen Green_PG' 'Kristaps Porzingis_PF'
 'Aaron Nesmith_SF' 'Kelly Olynyk_C' 'Christian Braun_SF'
 'Keldon Johnson_SG' 'Luke Kennard_SG'] 

['Luka Doncic_SG' 'Paolo Banchero_SF' 'Lauri Markkanen_SF' 'Cam Thomas_SG'
 'Brandon Ingram_SG' 'Naz Reid_PF' 'Jalen Green_PG' 'Alexandre Sarr_PF'
 'Scoot Henderson_PG' 'Kevin Porter Jr._PG' 'Dyson Daniels_SG'
 'Peyton Watson_PF'] 

['Luka Doncic_SG' 'Jaren Jackson Jr._PF' 'Paolo Banchero_SF'
 'Brandon Ingram_PF' 'Zach LaVine_SG' 'Kristaps Porzingis_C'
 'Jalen Green_PG' 'Aaron Nesmith_PF' 'Jaden Ivey_PG'
 'Bennedict Mathurin_SF' "De'Anthony Melton_PG" 'Duncan Robinson_SG'] 

['Luka Doncic_SG' 'Dejounte Murray_SG' 'Ja Moran

In [32]:
for i in group_stats:
    print(i, "\n")

[2.16444148 0.33680556 0.36990408 0.36850153 0.48958333 0.19517544
 0.40447154] 

[2.16918423 0.44270833 0.42775779 0.35015291 0.38020833 0.10087719
 0.46747967] 

[2.22837622 0.39409722 0.42416067 0.34556575 0.40104167 0.15131579
 0.51219512] 

[2.25043063 0.43923611 0.44304556 0.33180428 0.39583333 0.14254386
 0.49796748] 

[2.26253287 0.43229167 0.40617506 0.4029052  0.453125   0.09649123
 0.47154472] 

[2.27873226 0.48090278 0.47691847 0.35091743 0.41666667 0.08991228
 0.46341463] 



In [33]:
with open("demo.txt", "w") as txt_file:
    for line in top_groups:
        txt_file.write(", ".join(line) + 3*"\n")

In [34]:
vars = ["3PM", "PPG", "AST", "STL", "BLK", "TO"]
fig = go.Figure()    
for i in range(group_stats.shape[0]):

    fig.add_trace(go.Scatterpolar(
        r = group_stats[i][1:],
        theta = vars,
        fill = "toself",
        name = str(i)
    ))
fig.update_layout(
        autosize=False,
    width=1400,
    height=1400,
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 1]
    )),
  showlegend=True,
  legend=dict(orientation="v")
)

fig.show()